# Keras TextVectorization을 활용한 텍스트 벡터화

## 개요
이 노트북에서는 TensorFlow/Keras의 TextVectorization 레이어를 사용하여 텍스트 벡터화를 구현합니다.
이전에 직접 구현한 벡터화와 달리, Keras의 내장 기능을 활용하여 더 효율적이고 최적화된 텍스트 처리를 수행합니다.

## 학습 목표
1. Keras TextVectorization 레이어의 사용법 이해
2. 커스텀 표준화 및 토큰화 함수 구현
3. 빈도 기반 어휘 사전 구축 과정 이해
4. 직접 구현한 벡터화와의 차이점 분석

## 주요 특징
- **효율성**: TensorFlow 기반의 최적화된 연산
- **유연성**: 커스텀 전처리 함수 지원
- **확장성**: 대용량 데이터 처리 가능
- **통합성**: 딥러닝 모델과 직접 연결 가능

## 1. 필요한 라이브러리 임포트

In [ ]:
import re
import string
import tensorflow as tf
from keras.layers import TextVectorization
import numpy as np

print(f"TensorFlow 버전: {tf.__version__}")
print(f"구두점 목록: {string.punctuation}")

## 2. 커스텀 전처리 함수 구현

TextVectorization에서 사용할 커스텀 표준화 및 토큰화 함수를 정의합니다.

In [ ]:
def custom_standardization_fn(text):
    """
    커스텀 표준화 함수
    
    Args:
        text: TensorFlow 문자열 텐서
        
    Returns:
        표준화된 텍스트 (소문자 + 구두점 제거)
    """
    # 1. 소문자로 변환
    lower_text = tf.strings.lower(text)
    # 2. 정규식을 사용하여 구두점 제거
    return tf.strings.regex_replace(lower_text, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(text):
    """
    커스텀 토큰화 함수
    
    Args:
        text: 표준화된 TensorFlow 문자열 텐서
        
    Returns:
        토큰화된 문자열 배열
    """
    return tf.strings.split(text)

# 함수 테스트
test_text = tf.constant("Hello, World! How are you?")
print(f"원본 텍스트: {test_text.numpy().decode('utf-8')}")
print(f"표준화 결과: {custom_standardization_fn(test_text).numpy().decode('utf-8')}")
print(f"토큰화 결과: {custom_split_fn(custom_standardization_fn(test_text)).numpy()}")

## 3. TextVectorization 레이어 생성 및 설정

In [7]:
# TextVectorization 레이어 생성
text_vectorization = TextVectorization(
    output_mode="int",  # 출력 모드: 정수 시퀀스
    standardize=custom_standardization_fn,  # 커스텀 표준화 함수
    split=custom_split_fn  # 커스텀 토큰화 함수
)

print("TextVectorization 레이어가 생성되었습니다.")
print(f"출력 모드: int (정수 시퀀스)")
print(f"표준화 함수: 커스텀 함수 사용")
print(f"토큰화 함수: 커스텀 함수 사용")

# TextVectorization 객체의 주요 속성 확인
print(f"\n=== TextVectorization 레이어 정보 ===")
print(f"레이어 이름: {text_vectorization.name}")
print(f"레이어 타입: {type(text_vectorization).__name__}")

TextVectorization 레이어가 생성되었습니다.
출력 모드: int (정수 시퀀스)
표준화 함수: 커스텀 함수 사용
토큰화 함수: 커스텀 함수 사용

=== TextVectorization 레이어 정보 ===
레이어 이름: text_vectorization_3
레이어 타입: TextVectorization


## 4. 데이터셋 준비 및 어휘 사전 구축

In [9]:
# 테스트 데이터셋 (자연어1.py와 동일한 데이터 + 오타 수정)
dataset = [
    "I write, erase, rewrite",  # reqrite -> rewrite 수정
    "Erase again, and then",
    "A poppy blooms",
    "Dog is pretty"
]

print("=== 학습 데이터셋 ===")
for i, text in enumerate(dataset, 1):
    print(f"{i}. {text}")

# 어휘 사전 구축 (adapt 과정)
text_vectorization.adapt(dataset)
print("\n어휘 사전 구축이 완료되었습니다.")

=== 학습 데이터셋 ===
1. I write, erase, rewrite
2. Erase again, and then
3. A poppy blooms
4. Dog is pretty

어휘 사전 구축이 완료되었습니다.


## 5. 어휘 사전 분석

In [10]:
# 어휘 사전 추출
vocabulary = text_vectorization.get_vocabulary()

print("=== 구축된 어휘 사전 ===")
print(f"총 어휘 개수: {len(vocabulary)}")
print("\n어휘 목록 (빈도순 정렬):")
for i, word in enumerate(vocabulary):
    if word == "":
        print(f"{i:2d}. '[PADDING]' (패딩용)")
    elif word == "[UNK]":
        print(f"{i:2d}. '[UNK]' (미등록 단어)")
    else:
        print(f"{i:2d}. '{word}'")

print("\n=== 특수 토큰 ===")
print("- 인덱스 0: 패딩용 빈 문자열")
print("- 인덱스 1: 미등록 단어 [UNK]")
print("- 인덱스 2~: 실제 단어들 (빈도 높은 순)")

=== 구축된 어휘 사전 ===
총 어휘 개수: 15

어휘 목록 (빈도순 정렬):
 0. '[PADDING]' (패딩용)
 1. '[UNK]' (미등록 단어)
 2. 'erase'
 3. 'write'
 4. 'then'
 5. 'rewrite'
 6. 'pretty'
 7. 'poppy'
 8. 'is'
 9. 'i'
10. 'dog'
11. 'blooms'
12. 'and'
13. 'again'
14. 'a'

=== 특수 토큰 ===
- 인덱스 0: 패딩용 빈 문자열
- 인덱스 1: 미등록 단어 [UNK]
- 인덱스 2~: 실제 단어들 (빈도 높은 순)


## 6. 텍스트 인코딩 테스트

In [11]:
# 인코딩 테스트 (원본 코드의 오타 포함)
test_text = "I wruite, rewrite, and still rewrite agian"  # 오타: wruite, agian
print(f"테스트 텍스트: '{test_text}'")

# 인코딩 수행
encoded = text_vectorization(test_text)
print(f"인코딩 결과: {encoded.numpy()}")

# 인코딩 과정 상세 분석
print("\n=== 인코딩 과정 상세 분석 ===")
standardized_text = custom_standardization_fn(tf.constant(test_text))
tokens = custom_split_fn(standardized_text)
print(f"표준화된 텍스트: '{standardized_text.numpy().decode('utf-8')}'")
print(f"토큰들: {[token.numpy().decode('utf-8') for token in tokens]}")

print("\n토큰별 인덱스 매핑:")
for i, token_bytes in enumerate(tokens.numpy()):
    token = token_bytes.decode('utf-8')
    if token in vocabulary:
        idx = vocabulary.index(token)
        print(f"'{token}' -> {idx} (등록된 단어)")
    else:
        print(f"'{token}' -> 1 (미등록 단어, [UNK]로 처리)")

테스트 텍스트: 'I wruite, rewrite, and still rewrite agian'
인코딩 결과: [ 9  1  5 12  1  5  1]

=== 인코딩 과정 상세 분석 ===
표준화된 텍스트: 'i wruite rewrite and still rewrite agian'
토큰들: ['i', 'wruite', 'rewrite', 'and', 'still', 'rewrite', 'agian']

토큰별 인덱스 매핑:
'i' -> 9 (등록된 단어)
'wruite' -> 1 (미등록 단어, [UNK]로 처리)
'rewrite' -> 5 (등록된 단어)
'and' -> 12 (등록된 단어)
'still' -> 1 (미등록 단어, [UNK]로 처리)
'rewrite' -> 5 (등록된 단어)
'agian' -> 1 (미등록 단어, [UNK]로 처리)


## 7. 텍스트 디코딩 구현

In [12]:
# 디코딩을 위한 인덱스-단어 매핑 딕셔너리 생성
decoded_vocab = dict(enumerate(vocabulary))

print("=== 인덱스-단어 매핑 사전 ===")
for idx, word in decoded_vocab.items():
    if word == "":
        print(f"{idx} -> '[PADDING]'")
    else:
        print(f"{idx} -> '{word}'")

# 디코딩 수행
encoded_sequence = encoded.numpy()
decoded_sentence = " ".join(decoded_vocab[int(i)] for i in encoded_sequence)

print(f"\n=== 디코딩 결과 ===")
print(f"인코딩된 시퀀스: {encoded_sequence}")
print(f"디코딩된 문장: '{decoded_sentence}'")

# 원본과 디코딩 결과 비교
print(f"\n=== 원본 vs 디코딩 비교 ===")
print(f"원본 텍스트:     '{test_text}'")
print(f"표준화 텍스트:   '{standardized_text.numpy().decode('utf-8')}'")
print(f"디코딩 결과:     '{decoded_sentence}'")

=== 인덱스-단어 매핑 사전 ===
0 -> '[PADDING]'
1 -> '[UNK]'
2 -> 'erase'
3 -> 'write'
4 -> 'then'
5 -> 'rewrite'
6 -> 'pretty'
7 -> 'poppy'
8 -> 'is'
9 -> 'i'
10 -> 'dog'
11 -> 'blooms'
12 -> 'and'
13 -> 'again'
14 -> 'a'

=== 디코딩 결과 ===
인코딩된 시퀀스: [ 9  1  5 12  1  5  1]
디코딩된 문장: 'i [UNK] rewrite and [UNK] rewrite [UNK]'

=== 원본 vs 디코딩 비교 ===
원본 텍스트:     'I wruite, rewrite, and still rewrite agian'
표준화 텍스트:   'i wruite rewrite and still rewrite agian'
디코딩 결과:     'i [UNK] rewrite and [UNK] rewrite [UNK]'


## 8. 전체 데이터셋 처리 결과

In [13]:
print("=== 전체 데이터셋 인코딩/디코딩 결과 ===")

for i, text in enumerate(dataset, 1):
    # 인코딩
    encoded = text_vectorization(text)
    encoded_list = encoded.numpy()
    
    # 디코딩
    decoded = " ".join(decoded_vocab[int(idx)] for idx in encoded_list)
    
    # 표준화된 텍스트
    standardized = custom_standardization_fn(tf.constant(text)).numpy().decode('utf-8')
    
    print(f"\n{i}. 원본: '{text}'")
    print(f"   표준화: '{standardized}'")
    print(f"   인코딩: {encoded_list}")
    print(f"   디코딩: '{decoded}'")

=== 전체 데이터셋 인코딩/디코딩 결과 ===

1. 원본: 'I write, erase, rewrite'
   표준화: 'i write erase rewrite'
   인코딩: [9 3 2 5]
   디코딩: 'i write erase rewrite'

2. 원본: 'Erase again, and then'
   표준화: 'erase again and then'
   인코딩: [ 2 13 12  4]
   디코딩: 'erase again and then'

3. 원본: 'A poppy blooms'
   표준화: 'a poppy blooms'
   인코딩: [14  7 11]
   디코딩: 'a poppy blooms'

4. 원본: 'Dog is pretty'
   표준화: 'dog is pretty'
   인코딩: [10  8  6]
   디코딩: 'dog is pretty'


## 9. 다양한 출력 모드 비교

TextVectorization의 다른 출력 모드들을 살펴봅니다.

In [14]:
# 다른 출력 모드들 테스트
test_sentence = "I write erase rewrite"

# 1. Binary 모드 (단어 존재 여부)
vectorizer_binary = TextVectorization(
    output_mode="binary",
    standardize=custom_standardization_fn,
    split=custom_split_fn
)
vectorizer_binary.adapt(dataset)

# 2. Count 모드 (단어 빈도)
vectorizer_count = TextVectorization(
    output_mode="count",
    standardize=custom_standardization_fn,
    split=custom_split_fn
)
vectorizer_count.adapt(dataset)

# 3. TF-IDF 모드
vectorizer_tfidf = TextVectorization(
    output_mode="tf_idf",
    standardize=custom_standardization_fn,
    split=custom_split_fn
)
vectorizer_tfidf.adapt(dataset)

print(f"테스트 문장: '{test_sentence}'")
print(f"어휘 사전: {vocabulary}")
print("\n=== 다양한 출력 모드 비교 ===")
print(f"Int 모드:    {text_vectorization(test_sentence).numpy()}")
print(f"Binary 모드: {vectorizer_binary(test_sentence).numpy()}")
print(f"Count 모드:  {vectorizer_count(test_sentence).numpy()}")
print(f"TF-IDF 모드: {vectorizer_tfidf(test_sentence).numpy()}")

테스트 문장: 'I write erase rewrite'
어휘 사전: ['', '[UNK]', 'erase', 'write', 'then', 'rewrite', 'pretty', 'poppy', 'is', 'i', 'dog', 'blooms', 'and', 'again', 'a']

=== 다양한 출력 모드 비교 ===
Int 모드:    [9 3 2 5]
Binary 모드: [0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
Count 모드:  [0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
TF-IDF 모드: [0.         0.84729785 1.0986123  0.         1.0986123  0.
 0.         0.         1.0986123  0.         0.         0.
 0.         0.        ]


## 10. 미등록 단어 처리 및 어휘 크기 제한

In [16]:
# 어휘 크기를 제한한 벡터화기
limited_vectorizer = TextVectorization(
    max_tokens=5,  # 최대 5개 토큰만 유지 (패딩, UNK 포함)
    output_mode="int",
    standardize=custom_standardization_fn,
    split=custom_split_fn
)

limited_vectorizer.adapt(dataset)
limited_vocab = limited_vectorizer.get_vocabulary()

print("=== 어휘 크기 제한 테스트 ===")
print(f"제한된 어휘 사전 (최대 5개): {limited_vocab}")
print(f"어휘 개수: {len(limited_vocab)}")

# 제한된 어휘로 인코딩 테스트
test_with_unknown = "I love programming and coding"
print(f"\n테스트 문장: '{test_with_unknown}'")
encoded_limited = limited_vectorizer(test_with_unknown)
print(f"제한된 어휘로 인코딩: {encoded_limited.numpy()}")

# 디코딩
limited_decoded_vocab = dict(enumerate(limited_vocab))
decoded_limited = " ".join(limited_decoded_vocab[int(i)] for i in encoded_limited.numpy())
print(f"디코딩 결과: '{decoded_limited}'")

=== 어휘 크기 제한 테스트 ===
제한된 어휘 사전 (최대 5개): ['', '[UNK]', 'erase', 'write', 'then']
어휘 개수: 5

테스트 문장: 'I love programming and coding'
제한된 어휘로 인코딩: [1 1 1 1 1]
디코딩 결과: '[UNK] [UNK] [UNK] [UNK] [UNK]'


## 11. 성능 및 효율성 분석

In [17]:
import time

# 대용량 데이터 시뮬레이션
large_dataset = dataset * 1000  # 4,000개 문장
test_texts = ["I write code every day"] * 1000

print("=== 성능 테스트 ===")
print(f"테스트 데이터 크기: {len(test_texts)} 문장")

# TextVectorization 성능 측정
start_time = time.time()
for text in test_texts:
    _ = text_vectorization(text)
keras_time = time.time() - start_time

print(f"\nKeras TextVectorization 처리 시간: {keras_time:.4f}초")
print(f"초당 처리량: {len(test_texts)/keras_time:.1f} 문장/초")

# 배치 처리 성능
start_time = time.time()
batch_result = text_vectorization(test_texts[:100])  # 배치로 100개 처리
batch_time = time.time() - start_time

print(f"\n배치 처리 시간 (100개): {batch_time:.4f}초")
print(f"배치 처리 결과 shape: {batch_result.shape}")

=== 성능 테스트 ===
테스트 데이터 크기: 1000 문장

Keras TextVectorization 처리 시간: 5.2769초
초당 처리량: 189.5 문장/초

배치 처리 시간 (100개): 0.0115초
배치 처리 결과 shape: (100, 5)


## 12. 직접 구현 vs Keras TextVectorization 비교

In [18]:
print("=== 직접 구현 vs Keras TextVectorization 비교 ===")

comparison_data = [
    ["특징", "직접 구현 (MyVectorize)", "Keras TextVectorization"],
    ["구현 복잡도", "높음 (모든 기능 직접 구현)", "낮음 (내장 기능 활용)"],
    ["성능", "파이썬 기반, 상대적으로 느림", "TensorFlow 최적화, 빠름"],
    ["메모리 효율성", "기본적인 수준", "매우 효율적"],
    ["배치 처리", "별도 구현 필요", "내장 지원"],
    ["GPU 가속", "지원하지 않음", "완전 지원"],
    ["어휘 정렬", "삽입 순서", "빈도 기반 자동 정렬"],
    ["출력 모드", "정수 시퀀스만", "int, binary, count, tf-idf"],
    ["모델 통합", "별도 연결 작업 필요", "Keras 레이어로 직접 연결"],
    ["커스텀 함수", "클래스 메서드로 구현", "TensorFlow 함수로 구현"]
]

for row in comparison_data:
    print(f"{row[0]:<15} | {row[1]:<25} | {row[2]}")

print("\n=== 사용 권장 사항 ===")
print("✅ Keras TextVectorization 권장 상황:")
print("  - 딥러닝 모델과 통합 사용")
print("  - 대용량 데이터 처리")
print("  - 성능이 중요한 운영 환경")
print("  - 다양한 벡터화 방식 필요")

print("\n✅ 직접 구현 권장 상황:")
print("  - 학습 및 이해 목적")
print("  - 매우 특수한 전처리 로직")
print("  - 외부 의존성 최소화")
print("  - 완전한 제어가 필요한 경우")

=== 직접 구현 vs Keras TextVectorization 비교 ===
특징              | 직접 구현 (MyVectorize)       | Keras TextVectorization
구현 복잡도          | 높음 (모든 기능 직접 구현)          | 낮음 (내장 기능 활용)
성능              | 파이썬 기반, 상대적으로 느림          | TensorFlow 최적화, 빠름
메모리 효율성         | 기본적인 수준                   | 매우 효율적
배치 처리           | 별도 구현 필요                  | 내장 지원
GPU 가속          | 지원하지 않음                   | 완전 지원
어휘 정렬           | 삽입 순서                     | 빈도 기반 자동 정렬
출력 모드           | 정수 시퀀스만                   | int, binary, count, tf-idf
모델 통합           | 별도 연결 작업 필요               | Keras 레이어로 직접 연결
커스텀 함수          | 클래스 메서드로 구현               | TensorFlow 함수로 구현

=== 사용 권장 사항 ===
✅ Keras TextVectorization 권장 상황:
  - 딥러닝 모델과 통합 사용
  - 대용량 데이터 처리
  - 성능이 중요한 운영 환경
  - 다양한 벡터화 방식 필요

✅ 직접 구현 권장 상황:
  - 학습 및 이해 목적
  - 매우 특수한 전처리 로직
  - 외부 의존성 최소화
  - 완전한 제어가 필요한 경우


## 13. 실제 딥러닝 모델과의 통합 예제

In [19]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# 간단한 텍스트 분류 모델 구성 예제
def create_text_classification_model():
    """
    TextVectorization이 포함된 텍스트 분류 모델 생성
    """
    model = Sequential([
        text_vectorization,  # 전처리 레이어로 직접 사용
        Embedding(input_dim=len(vocabulary), output_dim=16),
        LSTM(32),
        Dense(1, activation='sigmoid')
    ])
    
    return model

# 모델 생성 및 구조 확인
model = create_text_classification_model()
print("=== 텍스트 분류 모델 구조 ===")
model.summary()

# 모델에 직접 원시 텍스트 입력 가능
sample_input = ["I love this movie", "This is terrible"]
print(f"\n원시 텍스트 입력: {sample_input}")

# 모델의 첫 번째 레이어(TextVectorization) 출력 확인
vectorized_output = text_vectorization(sample_input)
print(f"벡터화 출력: {vectorized_output.numpy()}")

print("\n✅ 장점: 원시 텍스트를 모델에 직접 입력 가능")
print("✅ 장점: 전처리와 모델이 하나의 파이프라인으로 통합")
print("✅ 장점: 모델 저장/로드 시 전처리 로직도 함께 저장")

=== 텍스트 분류 모델 구조 ===
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_3 (Text  (None, None)              0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, None, 16)          240       
                                                                 
 lstm (LSTM)                 (None, 32)                6272      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 6545 (25.57 KB)
Trainable params: 6545 (25.57 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

원시 텍스트 입력: ['I love this movie', 'This is terrible']
벡터화 출력: [[9 1

## 14. 결론 및 정리

### Keras TextVectorization의 주요 특징

1. **효율성과 성능**
   - TensorFlow 백엔드 기반의 최적화된 연산
   - GPU 가속 지원으로 대용량 데이터 처리 가능
   - 배치 처리 내장 지원

2. **유연성과 확장성**
   - 커스텀 전처리 함수 지원
   - 다양한 출력 모드 (int, binary, count, tf-idf)
   - 어휘 크기 제한 및 빈도 기반 필터링

3. **통합성**
   - Keras 레이어로서 모델에 직접 통합
   - 원시 텍스트 입력 지원
   - 모델 저장/로드 시 전처리 로직 보존

4. **자동화된 최적화**
   - 빈도 기반 어휘 정렬
   - 메모리 효율적인 구현
   - 프로덕션 환경 최적화

### 실제 사용 시 고려사항

- **어휘 크기 관리**: `max_tokens` 파라미터로 메모리 사용량 제어
- **시퀀스 길이**: `output_sequence_length`로 고정 길이 출력 가능
- **특수 토큰**: 패딩과 UNK 토큰의 적절한 활용
- **전처리 일관성**: 학습과 추론 시 동일한 전처리 보장

### 다음 단계 학습 방향

1. **서브워드 토큰화**: BPE, SentencePiece 등 고급 토큰화 기법
2. **다국어 처리**: 언어별 특수 전처리 요구사항
3. **대화형 모델**: 시퀀스-투-시퀀스 모델에서의 활용
4. **트랜스포머**: BERT, GPT 등 최신 모델과의 통합

## 15. 추가 실습 과제

다음 과제들을 통해 TextVectorization 활용 능력을 향상시켜보세요:

### 기초 과제
1. 한국어 텍스트로 동일한 과정 수행하기
2. 다양한 `max_tokens` 값에 따른 성능 변화 분석
3. 각 출력 모드별 특성과 사용 사례 정리

### 중급 과제
4. 영화 리뷰 데이터로 감성 분석 모델 구축
5. 시퀀스 길이 고정 기능 활용한 배치 처리 최적화
6. 커스텀 전처리 함수로 특수한 텍스트 정제 로직 구현

### 고급 과제
7. 대용량 텍스트 데이터셋으로 성능 벤치마킹
8. 다중 언어 텍스트 처리 파이프라인 구축
9. 실시간 텍스트 분류 서비스 구현